In [ ]:
import cv2
import os
import numpy as np
import mediapipe as mp
from scipy.io import loadmat
from sklearn.model_selection import train_test_split

In [ ]:
desired_height = 128
desired_width = 128
TRIANGLE_SIZE = 100
VIDEO_OUTPUT_FILE = 'output_with_triangle.avi'
def load_data(image_folder, mat_folder):
    print("Loading data...")
    images, poses = [], []
    image_files = sorted([f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png'))])
    mat_files = sorted([f for f in os.listdir(mat_folder) if f.endswith('.mat')])

    for image_file, mat_file in zip(image_files, mat_files):
        image_path = os.path.join(image_folder, image_file)
        mat_path = os.path.join(mat_folder, mat_file)
        
        image = cv2.imread(image_path)
        if image is None:
            print(f"Could not load image {image_file}. Skipping...")
            continue
        
        mat_data = loadmat(mat_path)
        
        if 'Pose_Para' not in mat_data:
            print(f"'Pose_Para' key not found in {mat_file}. Skipping...")
            continue
        
        ground_truth_poses = mat_data['Pose_Para']  
        poses.append(ground_truth_poses.flatten()) 
        
        images.append(image)

    print(f"Loaded {len(images)} images and {len(poses)} pose data.")
    return images, np.array(poses)


In [ ]:
def split_data(images, poses, test_size=0.2):
    return train_test_split(images, poses, test_size=test_size)


In [ ]:
def estimate_pose(image, landmarks):
    model_points = np.array([
        (0.0, 0.0, 0.0),           
        (0.0, -330.0, -65.0),      
        (-225.0, 170.0, -135.0),   
        (225.0, 170.0, -135.0),    
        (-150.0, -150.0, -125.0),  
        (150.0, -150.0, -125.0)    
    ])
    
    image_points = np.array([
        (landmarks[1].x * image.shape[1], landmarks[1].y * image.shape[0]),     
        (landmarks[152].x * image.shape[1], landmarks[152].y * image.shape[0]), 
        (landmarks[33].x * image.shape[1], landmarks[33].y * image.shape[0]),   
        (landmarks[263].x * image.shape[1], landmarks[263].y * image.shape[0]), 
        (landmarks[61].x * image.shape[1], landmarks[61].y * image.shape[0]),   
        (landmarks[291].x * image.shape[1], landmarks[291].y * image.shape[0])  
    ], dtype="double")

    focal_length = image.shape[1]  
    center = (image.shape[1] / 2, image.shape[0] / 2)
    camera_matrix = np.array(
        [[focal_length, 0, center[0]],
         [0, focal_length, center[1]],
         [0, 0, 1]], dtype="double"
    )

    dist_coeffs = np.zeros((4, 1)) 

    success, rotation_vector, translation_vector = cv2.solvePnP(
        model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE
    )

    if not success:
        raise ValueError("Pose estimation failed")

    rotation_matrix, _ = cv2.Rodrigues(rotation_vector)

    sy = np.sqrt(rotation_matrix[0, 0] * rotation_matrix[0, 0] + rotation_matrix[1, 0] * rotation_matrix[1, 0])
    singular = sy < 1e-6

    if not singular:
        x_angle = np.arctan2(rotation_matrix[2, 1], rotation_matrix[2, 2])
        y_angle = np.arctan2(-rotation_matrix[2, 0], sy)
        z_angle = np.arctan2(rotation_matrix[1, 0], rotation_matrix[0, 0])
    else:
        x_angle = np.arctan2(-rotation_matrix[1, 2], rotation_matrix[1, 1])
        y_angle = np.arctan2(-rotation_matrix[2, 0], sy)
        z_angle = 0

    pitch = np.degrees(x_angle)
    yaw = np.degrees(y_angle)
    roll = np.degrees(z_angle)

    return pitch, yaw, roll



In [ ]:
def preprocess_image(image):
    try:
        image_resized = cv2.resize(image, (desired_width, desired_height))
        return image_resized
    except cv2.error as e:
        print("Error resizing image:", e)
        return None

In [ ]:
def calculate_metrics(predictions, ground_truths):
    predictions = np.array(predictions)
    ground_truths = np.array(ground_truths)
    
    mae = np.mean(np.abs(predictions - ground_truths), axis=0)
    rmse = np.sqrt(np.mean((predictions - ground_truths) ** 2, axis=0))
    
    return mae, rmse

In [ ]:
def draw_pose_lines(frame, pitch, yaw, roll):
    # Define the center of the image
    height, width = frame.shape[:2]
    center_x, center_y = width // 2, height // 2
    
    # Define line length
    line_length = 1000

    # Calculate end points of the lines based on pitch, yaw, and roll
    pitch_end = (center_x, center_y - int(line_length * np.sin(np.radians(pitch))))
    yaw_end = (center_x + int(line_length * np.sin(np.radians(yaw))), center_y)
    roll_end = (center_x, center_y + int(line_length * np.sin(np.radians(roll))))
    
    # Draw lines on the frame
    cv2.line(frame, (center_x, center_y), pitch_end, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.line(frame, (center_x, center_y), yaw_end, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.line(frame, (center_x, center_y), roll_end, (0, 0, 255), 2, cv2.LINE_AA)
    
    # Add text for pitch, yaw, and roll
    cv2.putText(frame, f"Pitch: {pitch:.2f}", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(frame, f"Yaw: {yaw:.2f}", (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
    cv2.putText(frame, f"Roll: {roll:.2f}", (30, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

In [ ]:
def process_data(images, poses):
    mp_face_mesh = mp.solutions.face_mesh

    all_predictions = []
    all_ground_truths = []

    for idx, (image, ground_truth) in enumerate(zip(images, poses)):
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        with mp_face_mesh.FaceMesh(static_image_mode=True) as face_mesh:
            results = face_mesh.process(image_rgb)

            if results.multi_face_landmarks:
                face_landmarks = results.multi_face_landmarks[0]
                landmarks = face_landmarks.landmark

                try:
                    pitch, yaw, roll = estimate_pose(image, landmarks)
                    pose_estimation = np.array([pitch, yaw, roll])
                    
                    if ground_truth.shape[0] >= 3:
                        all_predictions.append(pose_estimation)
                        all_ground_truths.append(ground_truth[:3])

                    print(f"Processing image {idx}...")
                    print(f"Ground truth shape: {ground_truth.shape}")
                    print(f"Pose estimation shape: {pose_estimation.shape}")
                    print(f"Pose estimation: Pitch: {pitch}, Yaw: {yaw}, Roll: {roll}")

                except ValueError as e:
                    print(f"Error processing image {idx}: {e}")

            else:
                print(f"No face detected in image {idx}.")

    all_predictions = np.array(all_predictions)
    all_ground_truths = np.array(all_ground_truths)

    mae, rmse = calculate_metrics(all_predictions, all_ground_truths)
    if mae is not None and rmse is not None:
        print(f"Mean Absolute Error (MAE): {mae}")
        print(f"Root Mean Squared Error (RMSE): {rmse}")


In [ ]:
def test_with_camera():
    mp_face_mesh = mp.solutions.face_mesh
    camera_indices = [0, 1]  # Try different camera indices
    cap = None

    # Try different camera indices to find an accessible one
    for index in camera_indices:
        cap = cv2.VideoCapture(index)
        if cap.isOpened():
            print(f"Camera index {index} is accessible.")
            break
        else:
            print(f"Camera index {index} is not accessible.")

    if not cap or not cap.isOpened():
        print("Error: Camera not accessible.")
        return

    out = cv2.VideoWriter(VIDEO_OUTPUT_FILE, cv2.VideoWriter_fourcc(*'XVID'), 30, (int(cap.get(3)), int(cap.get(4))))

    with mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True) as face_mesh:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print("Error: Cannot read frame from camera.")
                break

            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(image_rgb)

            if results.multi_face_landmarks:
                face_landmarks = results.multi_face_landmarks[0]
                landmarks = face_landmarks.landmark

                try:
                    pitch, yaw, roll = estimate_pose(frame, landmarks)
                    draw_pose_lines(frame, pitch, yaw, roll)
                except ValueError as e:
                    print(f"Error processing frame: {e}")

            out.write(frame)
            cv2.imshow('Camera', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    image_folder = '/home/ahmed/Documents/GitHub/NTI-face-pose-statment-1/DataSet'
    mat_folder = '/home/ahmed/Documents/GitHub/NTI-face-pose-statment-1/DataSet'
    images, poses = load_data(image_folder, mat_folder)
    train_images, test_images, train_poses, test_poses = split_data(images, poses)

    print("Processing training data...")
    process_data(train_images, train_poses)

    print("Processing testing data...")
    process_data(test_images, test_poses)

    print("Testing with camera...")
    test_with_camera()